In [ ]:
# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# CIFAR-10 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# CIFAR-10 데이터셋 클래스 이름 정의
class_names = ['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭']

# 픽셀 정규화
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 레이블 원-핫 인코딩
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,        # 최대 20도까지 이미지 회전
    width_shift_range=0.2,    # 가로로 20% 이동
    height_shift_range=0.2,   # 세로로 20% 이동
    horizontal_flip=True,     # 이미지 좌우 반전
    zoom_range=0.1,           # 이미지 확대/축소
    shear_range=0.1           # 이미지 기울이기
)

datagen.fit(x_train)

# VGGNet 모델 정의
model = models.Sequential()

# 첫 번째 블록
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

# Fully Connected Layers
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # 과적합 방지
model.add(layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=256),  # 증강된 데이터로 학습
    epochs=250,
    validation_data=(x_test, y_test)
)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"테스트 정확도: {test_acc}")

# 모델 저장
model.save('vgg_cifar10_model.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# evaluate() : 모델 평가 지표 반환
results =model.evaluate(x_test, y_test)
print('테스트 손실 :', results[0])
print('테스트 정확도 :', results[1])

# 학습 데이터셋과 검증 데이터셋의 정확도와 오차
accuracy =history.history['accuracy']
val_accuracy =history.history['val_accuracy']
loss =history.history['loss']
val_loss =history.history['val_loss']

epochs = np.arange(len(accuracy))

# 정확도 그래프
plt.plot(epochs, accuracy, "b--", label='train_accuracy')         # 학습 정확도 표시
plt.plot(epochs, val_accuracy, "b", label='validation _accuracy') # 검증 정확도 표시
plt.title('Training and Validation Accuracy')                     # 제목
plt.legend()                                                      # 범례
plt.grid()                                                        # 그리드(격자)
plt.xlabel('epochs')                                              # x축 레이블
plt.ylabel('accuracy')                                            # y축 레이블
plt.show()                                                        # 그래프 출력

# 오차 그래프
plt.plot(epochs, loss, "r--", label='train_loss')                 # 학습 오차 표시
plt.plot(epochs, val_loss, "r", label='validation_loss')          # 검증 오차 표시
plt.title('Training and Validation Loss')                         # 제목
plt.legend()                                                      # 범례
plt.grid()                                                        # 그리드(격자)
plt.xlabel('epochs')                                              # x축 레이블
plt.ylabel('loss')                                                # y축 레이블
plt.show()                                                        # 그래프 출력

In [ ]:
model.save('vgg_cifar10_model.keras')